In [3]:
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [36]:
#load data
df_financial=pd.read_csv('Annual financial statements.csv')
df_comments=pd.read_csv('Tweet.csv')
df_stock=pd.read_csv('TSLA.csv')

In [37]:
def preprocess_comments(df, column_name):
    df=df[['post_date',column_name]].sample(100000,random_state=42)
    df['post_date']=pd.to_datetime(df['post_date'], unit='s').dt.date
    # 获取英语停用词列表
    stop_words = set(stopwords.words('english'))
    # 预处理函数
    def preprocess_text(text):
        # 转换为小写
        text = text.lower()
        # 使用正则表达式去除非字母字符
        text = re.sub(r'[^a-z]', ' ', text)
        # 去除多余空格
        text = re.sub(r'\s+', ' ', text).strip()
        # 分词
        words = word_tokenize(text)
        # 去除停用词并进行词形还原
        cleaned_text = ' '.join([word for word in words if word not in stop_words])
        return cleaned_text
    # 应用预处理函数到指定列
    df[column_name] = df[column_name].apply(preprocess_text)
    df=df.sort_values(by='post_date').reset_index(drop=True)
    return df

def preprocess_financial(df):
    df=df.dropna()
    df=df.loc[1:]
    return df

def preprocess_stock(df):
    df=df.dropna()
    return df


In [38]:
df_comments2 = preprocess_comments(df_comments, 'body')
df_financial2 = preprocess_financial(df_financial)
df_stock2 = preprocess_stock(df_stock)
df_comments2=df_comments2[df_comments2['post_date']>pd.to_datetime('2015')]

C:\Users\25381\AppData\Local\Temp\ipykernel_18616\4136369028.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_comments2=df_comments2[df_comments2['post_date']>pd.to_datetime('2015')]


In [39]:
def cal_emo(df):
    #Calculate emotional coefficient
    neu_ls=[]
    pos_ls=[]
    neg_ls=[]
    com_ls=[]
    for i in df['body'].tolist():
        i=str(i)
        sid = SentimentIntensityAnalyzer()
        emotion_dict= sid.polarity_scores(i)
        neu_ls.append(emotion_dict['neu'])
        pos_ls.append(emotion_dict['pos'])
        neg_ls.append(emotion_dict['neg'])
        com_ls.append(emotion_dict['compound'])
    df['neu']=neu_ls
    df['pos']=pos_ls
    df['neg']=neg_ls
    df['compound']=com_ls
    return df
df_comments3=cal_emo(df_comments2.sample(1000,random_state=42))
df_comments3

,post_date,body,neu,pos,neg,compound
33978,2016-08-22,google nexus nexus get official android nougat...,1.000,0.000,0.000,0.0000
59323,2018-01-09,stay ahead nasdaq news views analysis msft aap...,1.000,0.000,0.000,0.0000
35866,2016-09-14,download install google upcoming pixel launche...,1.000,0.000,0.000,0.0000
9747,2015-06-18,owensassetmgt may aapl realized stupid moving ...,0.638,0.000,0.362,-0.5267
32432,2016-07-28,amazon dbs looks cloud become fintech like rea...,0.741,0.259,0.000,0.4939
...,...,...,...,...,...,...
28270,2016-05-20,google data sheet friday may read http owler u...,1.000,0.000,0.000,0.0000
87670,2019-05-01,unfortunately tsla expenses grew bigger rate t...,0.745,0.000,0.255,-0.3400
87342,2019-04-28,big earnings week coming huge names reporting ...,0.860,0.140,0.000,0.6369
49021,2017-05-17,stocks buy near week highs eric landis abbv am...,1.000,0.000,0.000,0.0000


In [40]:
df_comments4=df_comments3.groupby('post_date',as_index=False)[['neu', 'pos', 'neg', 'compound']].mean()
df_stock2['Date']=pd.to_datetime(df_stock2['Date'])
df_comments4['post_date']=pd.to_datetime(df_comments4['post_date'])

C:\Users\25381\AppData\Local\Temp\ipykernel_18616\4228699650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock2['Date']=pd.to_datetime(df_stock2['Date'])


In [41]:
df_merge=pd.merge(df_stock2,df_comments4,left_on='Date',right_on='post_date')
df_merge.drop('post_date',axis=1,inplace=True)
df_merge

,Date,Open,High,Low,Close*,Adj Close**,Volume,neu,pos,neg,compound
0,2019-12-30,28.59,28.60,27.28,27.65,27.65,189018975.0,0.7410,0.2590,0.000,0.680800
1,2019-12-27,29.00,29.02,28.41,28.69,28.69,149352405.0,0.6705,0.2575,0.072,0.434450
2,2019-12-24,27.89,28.36,27.51,28.35,28.35,120820800.0,0.8455,0.1085,0.046,0.252300
3,2019-12-17,25.27,25.70,25.06,25.27,25.27,127556625.0,0.7690,0.2310,0.000,0.710200
4,2019-12-16,24.17,25.57,24.17,25.43,25.43,273314100.0,1.0000,0.0000,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
604,2015-01-15,12.97,13.05,12.67,12.79,12.79,78247860.0,0.8200,0.1800,0.000,0.353025
605,2015-01-08,14.19,14.25,14.00,14.04,14.04,51637635.0,0.8210,0.0000,0.179,-0.299700
606,2015-01-07,14.23,14.32,13.99,14.06,14.06,44525850.0,0.8090,0.1910,0.000,0.510600
607,2015-01-06,14.00,14.28,13.61,14.09,14.09,93929040.0,0.7450,0.2550,0.000,0.670500


In [42]:
df_merge.to_csv('df_merge.csv',index=False)

In [43]:
df_merge

,Date,Open,High,Low,Close*,Adj Close**,Volume,neu,pos,neg,compound
0,2019-12-30,28.59,28.60,27.28,27.65,27.65,189018975.0,0.7410,0.2590,0.000,0.680800
1,2019-12-27,29.00,29.02,28.41,28.69,28.69,149352405.0,0.6705,0.2575,0.072,0.434450
2,2019-12-24,27.89,28.36,27.51,28.35,28.35,120820800.0,0.8455,0.1085,0.046,0.252300
3,2019-12-17,25.27,25.70,25.06,25.27,25.27,127556625.0,0.7690,0.2310,0.000,0.710200
4,2019-12-16,24.17,25.57,24.17,25.43,25.43,273314100.0,1.0000,0.0000,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
604,2015-01-15,12.97,13.05,12.67,12.79,12.79,78247860.0,0.8200,0.1800,0.000,0.353025
605,2015-01-08,14.19,14.25,14.00,14.04,14.04,51637635.0,0.8210,0.0000,0.179,-0.299700
606,2015-01-07,14.23,14.32,13.99,14.06,14.06,44525850.0,0.8090,0.1910,0.000,0.510600
607,2015-01-06,14.00,14.28,13.61,14.09,14.09,93929040.0,0.7450,0.2550,0.000,0.670500


In [48]:
df_financial2 = preprocess_financial(df_financial)
df_financial2['year']=pd.to_datetime(df_financial2['fiscalDateEnding']).dt.year
df_merge['year']=df_merge['Date'].dt.year
df_merge_group=df_merge.groupby('year',as_index=False).mean()
df_merge2=pd.merge(df_merge_group,df_financial2,on='year')
df_merge2

C:\Users\25381\AppData\Local\Temp\ipykernel_18616\3154556911.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_merge_group=df_merge.groupby('year',as_index=False).mean()


,year,Open,High,Low,Close*,Adj Close**,Volume,neu,pos,neg,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2015,15.237328,15.481221,15.000305,15.257863,15.257863,6.632480e+07,0.820193,0.143916,0.035899,...,627927000,1290000000,446239000,-633166000,364976000,1083704000,None,-2322323000,131000,131425000
1,2016,14.070519,14.294370,13.810296,14.051333,14.051333,7.012652e+07,0.843670,0.107370,0.048953,...,984211000,None,6022457000,84492000,1891449000,4752911000,None,-2997237000,161000,161561000
2,2017,20.777658,21.027387,20.455225,20.730180,20.730180,9.473961e+07,0.814189,0.129167,0.056670,...,796549000,None,8832285000,130565000,2442970000,4237242000,None,-4974299000,169000,168797000
3,2018,20.991101,21.420367,20.541651,20.987706,20.987706,1.337886e+08,0.766026,0.164489,0.069480,...,2567699000,None,8410490000,372000000,2710000000,4923000000,None,-5317832000,0,172603000
4,2019,18.388862,18.687561,18.068699,18.392439,18.392439,1.489272e+08,0.746098,0.168676,0.085189,...,228000000,None,11187000000,260000000,2691000000,6618000000,None,-6083000000,1000000,181000000


In [49]:
df_merge2.to_csv('df_merge2.csv',index=False)

In [50]:
df_merge2.columns

Index(['year', 'Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume', 'neu',
       'pos', 'neg',
       ...
       'currentLongTermDebt', 'longTermDebtNoncurrent',
       'shortLongTermDebtTotal', 'otherCurrentLiabilities',
       'otherNonCurrentLiabilities', 'totalShareholderEquity', 'treasuryStock',
       'retainedEarnings', 'commonStock', 'commonStockSharesOutstanding'],
      dtype='object', length=102)

In [53]:
df_merge2[['Close*', 'currentLongTermDebt', 'pos']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Close*               5 non-null      float64
 1   currentLongTermDebt  5 non-null      object 
 2   pos                  5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 160.0+ bytes
